# Day 22

In [1]:
with open('../inputs/adventofcode.com_2022_day_22_input.txt', 'r') as f:
    data = f.read().splitlines()

print(f'The board is {len(data)-2} (rows) x {len(data[0])} (cols).')

The board is 200 (rows) x 150 (cols).


## Puzzle 1

In [2]:
import numpy as np

rows = len(data)-2
cols = max([len(r) for r in data[:-2]])
print(rows, cols)
# The coding is: -1 = outside the board, 0 = tile, 1 = wall
board = np.ones((rows, cols), dtype=int) * (-1)
print(board.shape)

for y, row in enumerate(data[:-2]):
    print(row)
    for x, c in enumerate(row):
        if c == '.':
            board[y, x] = 0
        elif c == '#':
            board[y, x] = 1

print(board)

# Start at the leftmost empty tile in the top row
# 3d coord is the direction: 0 = right, 1 = down, 2 = left, 3 = up
start = (0, np.where(board[0] == 0)[0][0], 0)
print(start)

200 150
(200, 150)
                                                  ..................#....#...#..#...##.....#.....................##...........................#.#....#
                                                  #......#........##..##.#.........#.#..#...........#...#..........................#........#.#.....#.
                                                  ..................#..........#....#.##...........#..........#.......................................
                                                  .....#..........#...#...#...........#.#.......................#...........#...........#.............
                                                  ...........#...............#............#......#......#.#..#...#.#....................#.............
                                                  .............#..#......#...................#.#.#..#...##.....#..............#...#...#.....#.........
                                                  #...#....................

In [3]:
import re

pattern = re.compile(r'(\d+|[LR])')
movements = pattern.findall(data[-1])

pos = start

for m in movements:
    if m.isnumeric():
        m = int(m)
        for s in range(m):
            # Check if we can move in the direction we are facing
            row = pos[0]
            col = pos[1]
            dir = pos[2]
            if dir == 0:    # Facing right
                if pos[1] == cols-1 or board[row, pos[1]+1] == -1:
                    col = np.where(board[row] != -1)[0][0]
                else:
                    col = pos[1]+1
            elif dir == 1:  # Facing down
                if pos[0] == rows-1 or board[pos[0]+1, col] == -1:
                    row = np.where(board[:, col] != -1)[0][0]
                else:
                    row = pos[0]+1
            elif dir == 2:  # Facing left
                if pos[1] == 0 or board[row, pos[1]-1] == -1:
                    col = np.where(board[row] != -1)[0][-1]
                else:
                    col = pos[1]-1
            elif dir == 3:  # Facing up
                if pos[0] == 0 or board[pos[0]-1, col] == -1:
                    row = np.where(board[:, col] != -1)[0][-1]
                else:
                    row = pos[0]-1
                pass
            next = (row, col, dir)
            if board[row, col] == 1:
                break
            else:
                pos = next

    else:
        if m == 'L':
            pos = (pos[0], pos[1], (pos[2]-1)%4)
        elif m == 'R':
            pos = (pos[0], pos[1], (pos[2]+1)%4)

print(f'Final position: {pos[0]+1}, {pos[1]+1}, {pos[2]}')
password = 1000 * (pos[0]+1) + 4 * (pos[1]+1) + pos[2]
print(f'The password is {password}.')

Final position: 162, 46, 2
The password is 162186.


## Puzzle 2

In [4]:
cube_size = 50
grid2side = np.zeros((board.shape[0] // cube_size, board.shape[1] // cube_size), dtype=int)

num_side = 1
for row in range(board.shape[0] // cube_size):
    for col in range(board.shape[1] // cube_size):
        cube_side = board[row*cube_size:(row+1)*cube_size, col*cube_size:(col+1)*cube_size]
        if (cube_side >= 0).any():
            grid2side[row, col] = num_side
            num_side += 1

print(grid2side)


[[0 1 2]
 [0 3 0]
 [4 5 0]
 [6 0 0]]


In [5]:
# To define the connections, draw on a paper the flattened cube, then figure out the sides that are touching
# Tip: start by the sides that are at 90 degrees from each other, and work your way around the cube
# This dictionary must contain for each side from the previous cell, for each exterior side (as key) the connecting side and direction (as value).

connections = {}

connections[1] = {'top': (6, 'left'), 'left': (4, 'left')}      # Example: side one has a top side that connects to side 6 on the left, and a left side that connects to side 4 on the left
connections[2] = {'top': (6, 'bottom'), 'right': (5, 'right'), 'bottom': (3, 'right')}
connections[3] = {'left': (4, 'top'), 'right': (2, 'bottom')}
connections[4] = {'top': (3, 'left'), 'left': (1, 'left')}
connections[5] = {'right': (2, 'right'), 'bottom': (6, 'right')}
connections[6] = {'left': (1, 'top'), 'bottom': (2, 'top'), 'right': (5, 'bottom')}

In [6]:
import re


def wrap_cube(row, col, d):
    cube_side_num = grid2side[row // cube_size, col // cube_size]
    # Facing right
    if d == 0:
        next_side_num, side = connections[cube_side_num]['right']
        next_side_coords = np.where(grid2side == next_side_num)
        next_side_coords = int(next_side_coords[0]), int(next_side_coords[1])
        
        row_offset = row - (row // cube_size) * cube_size
        if side == 'top':
            row = next_side_coords[0] * cube_size
            col = (next_side_coords[1]+1) * cube_size -1 - row_offset
            dir = 1
        elif side == 'right':
            row = (next_side_coords[0]+1) * cube_size -1 - row_offset
            col = (next_side_coords[1]+1) * cube_size -1
            dir = 2
        elif side == 'bottom':
            row = next_side_coords[0] * cube_size + cube_size-1
            col = next_side_coords[1] * cube_size + row_offset
            dir = 3
        elif side == 'left':
            row = next_side_coords[0] * cube_size + row_offset
            col = next_side_coords[1] * cube_size
            dir = 0

    # Facing down
    elif d == 1:
        next_side_num, side = connections[cube_side_num]['bottom'] 
        next_side_coords = np.where(grid2side == next_side_num)
        next_side_coords = int(next_side_coords[0]), int(next_side_coords[1])
        
        col_offset = col - (col // cube_size) * cube_size
        if side == 'right':
            row = next_side_coords[0] * cube_size + col_offset
            col = (next_side_coords[1]+1) * cube_size -1
            dir = 2
        elif side == 'left':
            row = (next_side_coords[0]+1) * cube_size -1 - col_offset
            col = next_side_coords[1] * cube_size
            dir = 0
        elif side == 'bottom':
            row = (next_side_coords[0]+1) * cube_size -1
            col = (next_side_coords[1]+1) * cube_size -1 - col_offset
            dir = 3
        elif side == 'top':
            row = next_side_coords[0] * cube_size
            col = next_side_coords[1] * cube_size + col_offset
            dir = 1

    # Facing left
    elif d == 2:
        next_side_num, side = connections[cube_side_num]['left'] 
        next_side_coords = np.where(grid2side == next_side_num)
        next_side_coords = int(next_side_coords[0]), int(next_side_coords[1])
        
        row_offset = row - (row // cube_size) * cube_size
        if side == 'top':
            row = next_side_coords[0] * cube_size
            col = next_side_coords[1] * cube_size + row_offset
            dir = 1
        elif side == 'left':
            row = (next_side_coords[0]+1) * cube_size -1 - row_offset
            col = next_side_coords[1] * cube_size
            dir = 0
        elif side == 'bottom':
            row = (next_side_coords[0]+1) * cube_size -1
            col = (next_side_coords[1]+1) * cube_size -1 - row_offset
            dir = 3
        elif side == 'right':
            row = next_side_coords[0] * cube_size + row_offset
            col = (next_side_coords[1]+1) * cube_size -1
            dir = 2

    # Facing up
    elif d == 3:
        next_side_num, side = connections[cube_side_num]['top'] # 2 top
        next_side_coords = np.where(grid2side == next_side_num)
        next_side_coords = int(next_side_coords[0]), int(next_side_coords[1])
        
        col_offset = col - (col // cube_size) * cube_size
        if side == 'top':
            row = next_side_coords[0] * cube_size
            col = next_side_coords[1] * cube_size + col_offset
            dir = 1
        elif side == 'right':
            row = (next_side_coords[0]+1) * cube_size -1 - col_offset
            col = (next_side_coords[1]+1) * cube_size -1
            dir = 2
        elif side == 'left':
            row = next_side_coords[0] * cube_size + col_offset
            col = next_side_coords[1] * cube_size
            dir = 0
        elif side == 'bottom':
            row = (next_side_coords[0]+1) * cube_size-1
            col = next_side_coords[1] * cube_size + col_offset
            dir = 3
        
    return (row, col, dir)

pattern = re.compile(r'(\d+|[LR])')
movements = pattern.findall(data[-1])

pos = start
print(f'Initial position {pos}')

for m in movements:
    if m.isnumeric():
        m = int(m)
        for s in range(m):
            # Check if we can move in the direction we are facing
            row = pos[0]
            col = pos[1]
            dir = pos[2]

            # Facing right
            if dir == 0:    
                if pos[1] == cols-1 or board[row, pos[1]+1] == -1:
                    # Wrap around
                    row, col, dir = wrap_cube(row, col, dir)
                else:
                    col = pos[1]+1

            # Facing down
            elif dir == 1:  
                if pos[0] == rows-1 or board[pos[0]+1, col] == -1:
                    row, col, dir = wrap_cube(row, col, dir)
                else:
                    row = pos[0]+1

            # Facing left
            elif dir == 2:  
                if pos[1] == 0 or board[row, pos[1]-1] == -1:
                    row, col, dir = wrap_cube(row, col, dir)
                else:
                    col = pos[1]-1

            # Facing up
            elif dir == 3:  
                if pos[0] == 0 or board[pos[0]-1, col] == -1:
                    row, col, dir = wrap_cube(row, col, dir)
                else:
                    row = pos[0]-1

            next = (row, col, dir)
            # If there is a wall, we cannot move
            if board[row, col] == 1:
                break
            else:
                # Update position
                pos = next
    else:
        if m == 'L':
            pos = (pos[0], pos[1], (pos[2]-1)%4)
        elif m == 'R':
            pos = (pos[0], pos[1], (pos[2]+1)%4)

print(f'Final position: {pos[0]+1}, {pos[1]+1}, {pos[2]}')
password = 1000 * (pos[0]+1) + 4 * (pos[1]+1) + pos[2]
print(f'The password is {password}.')

Initial position (0, 50, 0)
Final position: 55, 66, 3
The password is 55267.
